# Monty Hall Problem
This experiment simulates the Monty Hall problem to verify that the probability of winning given switching the chosen door is higher. For a more detailed explanation, please check out this [video](https://www.youtube.com/watch?v=4Lb-6rxZxx0).

In [1]:
from random import randint, choice

In the `MontyHall` class, you may choose to run `simulate_game` to automatically run the simulation across multiple trials. If you wish to try out the game yourself, you may execute the `play_game` function.

In [81]:
class MontyHall():
    def __init__(self, doors):
        self.doors = doors
        self.total_stays = 0
        self.total_changed = 0
        self.total_changed_wins = 0
        self.total_stay_wins = 0
        self.always_change = False
        self.verbose = True

    def _choose_remain(self, game):
        remain = randint(1, self.doors)
        if game['init'] != game['car']:
            remain = game['car']
        else:
            while remain == game['init']: remain = randint(1, self.doors)
        return remain
    
    def _verbose(self, game):
        print(f'Initial choice: {game["init"]}', end='; ')
        print(f'Remaining door: {game["remain"]}', end='; ')
        print(f'Final choice: {game["final"]}, Car: {game["car"]}', end='; ')

    def simulate_game(self, trials=1, always_change=False, verbose=True):
        self.trials = trials
        self.always_change = always_change
        self.verbose = verbose
        for trial in range(trials):
            game = {}
            game['car'] = randint(1, self.doors)
            game['init'] = randint(1, self.doors)
            game['remain'] = self._choose_remain(game)
            game['final'] = choice([game['init'], game['remain']]) if not always_change else game['remain']

            changed = True if game['final'] != game['init'] else False
            win = True if game['final'] == game['car'] else False
            changed_win = True if changed and win else False
            stay_win = True if (not changed) and win else False 

            self.total_stays += not changed
            self.total_changed += changed
            self.total_changed_wins += changed_win
            self.total_stay_wins += stay_win

            if verbose:
                print(f'Trial no. {trial}')
                self._verbose(game)
                print('Contestant wins' if win else 'Contestant loses')
                
        return {'Trials': self.trials, 'Stays': self.total_stays, 'Stay Wins': self.total_stay_wins, 'Changes': self.total_changed, 'Change Wins': self.total_changed_wins}
    
    def play_game(self, trials, verbose=True):
        self.trials = trials
        for trial in range(trials):
            game = {}
            game['car'] = randint(1, self.doors)
            game['init'] = int(input(f'Trial {trial}:\nChoose a door [1, 2, 3]: '))
            game['remain'] = self._choose_remain(game)
            final = str.lower(input(f'Opening all other doors except door {game["remain"]}. Switch [s] or Keep [k] your choice? '))
            assert final == 's' or final == 'k', 'Choice should either be \'s\' for Switch or \'k\' for Keep'
            game['final'] = game['init'] if final == 'k' else game['remain']
            
            changed = True if game['final'] != game['init'] else False
            win = True if game['final'] == game['car'] else False
            changed_win = True if changed and win else False
            stay_win = True if (not changed) and win else False 

            self.total_stays += not changed
            self.total_changed += changed
            self.total_changed_wins += changed_win
            self.total_stay_wins += stay_win

            if verbose:
                print(f'Trial no. {trial}')
                self._verbose(game)
                print('Contestant wins' if win else 'Contestant loses')

        self.print_results()

    def print_results(self):
        if self.verbose: print('')
        print(f'Results\nNo. of trials: {self.trials}; Total wins: {self.total_changed_wins + self.total_stay_wins}')
        print(f'Choices kept: {self.total_stays}; Kept wins: {self.total_stay_wins}; Probability of win given choice kept: {self.total_stay_wins*100/self.total_stays if self.total_stays else "N/A"}%')
        print(f'Choices switched: {self.total_changed}; Switched wins: {self.total_changed_wins}; Probability of win given choice switched: {self.total_changed_wins*100/self.total_changed if self.total_changed else "N/A"}%')

If we simulate the original Monty Hall problem with 3 doors over 10 trials, we see that the probability of winning given that you kept your choice is $25.00\%$, which is significantly lower than the probability of winning given that you switched choices ($66.67\%$).

In [85]:
sim = MontyHall(doors=3)
results = sim.simulate_game(trials=10)
sim.print_results()

Trial no. 0
Initial choice: 1; Remaining door: 2; Final choice: 2, Car: 2; Contestant wins
Trial no. 1
Initial choice: 2; Remaining door: 3; Final choice: 3, Car: 2; Contestant loses
Trial no. 2
Initial choice: 2; Remaining door: 3; Final choice: 3, Car: 3; Contestant wins
Trial no. 3
Initial choice: 3; Remaining door: 1; Final choice: 1, Car: 1; Contestant wins
Trial no. 4
Initial choice: 2; Remaining door: 3; Final choice: 2, Car: 3; Contestant loses
Trial no. 5
Initial choice: 1; Remaining door: 3; Final choice: 1, Car: 1; Contestant wins
Trial no. 6
Initial choice: 2; Remaining door: 3; Final choice: 3, Car: 2; Contestant loses
Trial no. 7
Initial choice: 3; Remaining door: 2; Final choice: 3, Car: 2; Contestant loses
Trial no. 8
Initial choice: 3; Remaining door: 1; Final choice: 3, Car: 1; Contestant loses
Trial no. 9
Initial choice: 3; Remaining door: 2; Final choice: 2, Car: 2; Contestant wins

Results
No. of trials: 10; Total wins: 5
Choices kept: 4; Kept wins: 1; Probability 

If we stretch the simulation to 10,000 trials, we see that the probability of winning given that you keep your choice converges to around $33\%$ and the probability of winning given that you switched your choice converges to $66\%$, which are the expected conditional probabilities of the problem.

In [86]:
sim = MontyHall(doors=3)
results = sim.simulate_game(trials=10000, verbose=False)
sim.print_results()

Results
No. of trials: 10000; Total wins: 4935
Choices kept: 5022; Kept wins: 1638; Probability of win given choice kept: 32.61648745519713%
Choices switched: 4978; Switched wins: 3297; Probability of win given choice switched: 66.23141824025713%


This code block can be used if you only want to simulate the switching of doors (the program always chooses to switch doors). We see that the probability still converges to around $66$ to $67\%$.

In [87]:
sim = MontyHall(doors=3)
results = sim.simulate_game(trials=10000, always_change=True, verbose=False)
sim.print_results()

Results
No. of trials: 10000; Total wins: 6716
Choices kept: 0; Kept wins: 0; Probability of win given choice kept: N/A%
Choices switched: 10000; Switched wins: 6716; Probability of win given choice switched: 67.16%


If we increase the number of doors to 100, we see that the difference in probabilities becomes even larger. Getting the door right on the first try is a $1/100$ chance, so the probability of the car being on the other doors is $99\%$. Since the host removes all other doors except one, then the probability of the car being on the other door is still $99\%$.

In [88]:
sim = MontyHall(doors=100)
results = sim.simulate_game(trials=10000, verbose=False)
sim.print_results()

Results
No. of trials: 10000; Total wins: 5000
Choices kept: 4998; Kept wins: 50; Probability of win given choice kept: 1.0004001600640255%
Choices switched: 5002; Switched wins: 4950; Probability of win given choice switched: 98.96041583366653%


If we choose to always switch choices, we win around $99\%$ of the time in 10,000 trials.

In [89]:
sim = MontyHall(doors=100)
results = sim.simulate_game(trials=10000, always_change=True, verbose=False)
sim.print_results()

Results
No. of trials: 10000; Total wins: 9905
Choices kept: 0; Kept wins: 0; Probability of win given choice kept: N/A%
Choices switched: 10000; Switched wins: 9905; Probability of win given choice switched: 99.05%


If you wish to play the game yourself, you may run this code block.

In [90]:
sim = MontyHall(doors=3)
sim.play_game(trials=5)

Trial no. 0
Initial choice: 1; Remaining door: 3; Final choice: 1, Car: 3; Contestant loses
Trial no. 1
Initial choice: 2; Remaining door: 1; Final choice: 2, Car: 2; Contestant wins
Trial no. 2
Initial choice: 3; Remaining door: 2; Final choice: 2, Car: 2; Contestant wins
Trial no. 3
Initial choice: 2; Remaining door: 3; Final choice: 3, Car: 3; Contestant wins
Trial no. 4
Initial choice: 1; Remaining door: 2; Final choice: 2, Car: 2; Contestant wins

Results
No. of trials: 5; Total wins: 4
Choices kept: 2; Kept wins: 1; Probability of win given choice kept: 50.0%
Choices switched: 3; Switched wins: 3; Probability of win given choice switched: 100.0%
